In [15]:
import torch
from torch import nn
import time


In [6]:
class Block(nn.Module):
    def __init__(self,in_channel,out_channel,dropout_rate):
        super(Block,self).__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(in_channel,out_channel,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        )
        
        self.layer2=nn.Sequential(
            nn.Conv2d(out_channel,out_channel,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        )

        self.dropout=nn.Dropout(dropout_rate)

    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.dropout(out)

        return out

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(1,64,kernel_size=5,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        )

        self.blk1=Block(64,64,0.2)
        self.blk2=Block(64,128,0.2)
        self.blk3=Block(128,256,0.2)
        self.blk4=Block(256,512,0.2)

        self.avgpool=nn.AvgPool2d(kernel_size=3,stride=1,padding=1)
        self.fc=nn.Linear(512*37*37,128)
        self.fc2=nn.Linear(128,2)
        self.fl=nn.Softmax(dim=1)

    def forward(self,x):
        out=self.conv1(x)

        out=self.blk1(out)
        out=self.blk2(out)
        out=self.blk3(out)
        out=self.blk4(out)

        out=self.avgpool(out)
        out=out.reshape(out.size(0), -1)
        out=self.fc(out)
        out=self.fc2(out)
        out=self.fl(out)

        return out

In [12]:
model=Classifier()
print(model)

Classifier(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (blk1): Block(
    (layer1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (layer2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (blk2): Block(
    (layer1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (layer2): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (blk3): Block(
    (layer1): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    

In [ ]:
def validation(model, validation_loader, criterion):
    validation_loss = 0
    accuracy = 0
    
    for images, labels in validation_loader:
                
        outputs = model(images)
        validation_loss += criterion(outputs, torch.max(labels,1)[1]).item()
        
        maxProb_idx = torch.max(outputs,1)[1]
        
        equality = torch.eq(torch.max(labels,1)[1],maxProb_idx).float()
        accuracy += torch.mean(equality)

    return validation_loss, accuracy

def train_accuracy(predicted_labels,target_labels):
    maxProb_idx = torch.max(predicted_labels,1)[1]
    
    equality = torch.eq(torch.max(target_labels,1)[1],maxProb_idx).float()
    # print("equality  ",equality)
    accuracy = torch.mean(equality)

    # print("predicted label  ",predicted_labels)
    # print("tarfet leable  ",target_labels)
    # print("accuracy   ",accuracy)
    return accuracy

In [ ]:
def train_model(train_loader,epochs,loss_function):
    model=Classifier()

    criterion=loss_function
    optimizer=optim.Adam(model.parameters(),lr=0.001)

    training_loss_list=[]
    training_accuracy_list=[]
    acc_list=[]
    val_acc_list=[]
    start_time=time.time()
    for epoch in range(epochs):
        model.train()
        for batch_idx, (images_data, target_labels) in enumerate(train_loader):
            print(batch_idx, '----------------------')
            # print("image data ",images_data)
            outputs=model(images_data)
            # print("output   ",outputs)
            loss=criterion(outputs,torch.max(target_labels,1)[1])
            training_loss_list.append(loss.item())
            accuracy=train_accuracy(outputs,target_labels)
            training_accuracy_list.append(accuracy)
            # print("loss ", training_loss_list)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            

            if (batch_idx+1)%10==0:
                model.eval()
                with torch.no_grad():
                    validation_loss,validation_accuracy=validation(model,val_loader,criterion=loss_function)

                print("Epoch: {}/{} @ {} ".format(epoch, epochs,time.time()),
                      "\n",
                      "Training Loss: {:.3f} - ".format(sum(training_loss_list)/len(training_loss_list)),
                      "Training Accuracy: {:.3f} - ".format(sum(training_accuracy_list)/len(training_accuracy_list)),
                      "Validation Loss: {:.3f} - ".format(validation_loss/len(val_loader)),
                      "Validation Accuracy: {:.3f}".format(validation_accuracy/len(val_loader)))
                
                acc_list.append(sum(training_accuracy_list)/len(training_accuracy_list))
                val_acc_list.append(validation_accuracy/len(val_loader))
                
                training_loss_list=[]
                training_accuracy_list=[]

                model.train()

    plt.plot(acc_list)
    plt.plot(val_acc_list)
    plt.show()

    print("Training finished!","\n","Run time: {:.3f} mins".format((time.time() - start_time)/60))
    return model